# 01 – Data Collection and Panel Construction

This notebook builds the raw and processed datasets used in the project:

- Sets up the project folder structure (`data/raw/`, `data/processed/`)
- Loads Eurostat, OECD (CCI), and Statcounter exports
- Cleans and harmonizes country codes and dates
- Constructs:
  - A monthly macro–social media panel (for H2/H3 analyses)
  - A final annual country × year panel (`panel_annual.csv`)

**Inputs (expected in `data/raw/`):**
- `tin00127_linear_2_0.csv` — Eurostat social media participation
- `une_rt_m_linear_2_0.csv` — Eurostat unemployment (monthly)
- `prc_hicp_midx__custom_19133241_linear.csv` — Eurostat HICP (inflation, part 1)
- `prc_hicp_midx__custom_19134087_linear.csv` — Eurostat HICP (inflation, part 2)
- `export-2025-11-26T13_03_48.204Z.csv` — OECD Consumer Confidence Index export
- `social_media-ww-monthly-200903-202511.csv` — Statcounter monthly platform shares

**Outputs (written to `data/processed/`):**
- `panel_monthly_h3.csv` — Monthly H3 panel (economic stress × entertainment vs professional)
- `panel_annual.csv` — Final country-year panel used in EDA, hypothesis tests, and ML.


## CELL 1: Project directory setup

----------------------------------------------
This cell automatically detects the project root
and initializes the directories for raw and processed data.

In [21]:

from pathlib import Path
import os

# Detect project root:
# If notebook is inside 'notebooks/', use its parent as project root.
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name.lower() in ["notebook", "notebooks"]:
    PROJECT_ROOT = PROJECT_ROOT.parent
elif not (PROJECT_ROOT / "data").exists():
    # If current directory isn't the project root, go one level up
    PROJECT_ROOT = PROJECT_ROOT.parent

# Define folder paths
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

# Create folders if they do not exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT   :", PROJECT_ROOT)
print("DATA_RAW       :", DATA_RAW)
print("DATA_PROCESSED :", DATA_PROCESSED)

PROJECT_ROOT   : /Users/ibrahimgozlukaya/Desktop/DSA210
DATA_RAW       : /Users/ibrahimgozlukaya/Desktop/DSA210/data/raw
DATA_PROCESSED : /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed


## CELL 2: Country mapping + Statcounter template URL

---------------------------------------------------
Statcounter provides data using a country slug and a region name.
We map ISO-2 country codes to Statcounter-compatible slugs.

In [2]:

tries = {
    "AL": {"slug": "albania", "name_in_url": "Albania"},
    "AT": {"slug": "austria", "name_in_url": "Austria"},
    "BA": {"slug": "bosnia-and-herzegovina", "name_in_url": "Bosnia and Herzegovina"},
    "BE": {"slug": "belgium", "name_in_url": "Belgium"},
    "BG": {"slug": "bulgaria", "name_in_url": "Bulgaria"},
    "CH": {"slug": "switzerland", "name_in_url": "Switzerland"},
    "CY": {"slug": "cyprus", "name_in_url": "Cyprus"},
    "CZ": {"slug": "czech-republic", "name_in_url": "Czech Republic"},
    "DE": {"slug": "germany", "name_in_url": "Germany"},
    "DK": {"slug": "denmark", "name_in_url": "Denmark"},
    "EE": {"slug": "estonia", "name_in_url": "Estonia"},
    "EL": {"slug": "greece", "name_in_url": "Greece"},
    "ES": {"slug": "spain", "name_in_url": "Spain"},
    "FI": {"slug": "finland", "name_in_url": "Finland"},
    "FR": {"slug": "france", "name_in_url": "France"},
    "HR": {"slug": "croatia", "name_in_url": "Croatia"},
    "HU": {"slug": "hungary", "name_in_url": "Hungary"},
    "IE": {"slug": "ireland", "name_in_url": "Ireland"},
    "IS": {"slug": "iceland", "name_in_url": "Iceland"},
    "IT": {"slug": "italy", "name_in_url": "Italy"},
    "LT": {"slug": "lithuania", "name_in_url": "Lithuania"},
    "LU": {"slug": "luxembourg", "name_in_url": "Luxembourg"},
    "LV": {"slug": "latvia", "name_in_url": "Latvia"},
    "ME": {"slug": "montenegro", "name_in_url": "Montenegro"},
    "MK": {"slug": "north-macedonia", "name_in_url": "North Macedonia"},
    "MT": {"slug": "malta", "name_in_url": "Malta"},
    "NL": {"slug": "netherlands", "name_in_url": "Netherlands"},
    "NO": {"slug": "norway", "name_in_url": "Norway"},
    "PL": {"slug": "poland", "name_in_url": "Poland"},
    "PT": {"slug": "portugal", "name_in_url": "Portugal"},
    "RO": {"slug": "romania", "name_in_url": "Romania"},
    "RS": {"slug": "serbia", "name_in_url": "Serbia"},
    "SE": {"slug": "sweden", "name_in_url": "Sweden"},
    "SI": {"slug": "slovenia", "name_in_url": "Slovenia"},
    "SK": {"slug": "slovakia", "name_in_url": "Slovakia"},
    "TR": {"slug": "turkey", "name_in_url": "Turkey"},
    "UK": {"slug": "united-kingdom", "name_in_url": "United Kingdom"},
    "XK": {"slug": "kosovo", "name_in_url": "Kosovo"},
}

# Template URL (Spain version). We dynamically replace Spain with each country.
template_url = (
    "https://gs.statcounter.com/social-media-stats/all/spain/chart.php"
    "?device=Desktop%20%26%20Mobile%20%26%20Tablet%20%26%20Console"
    "&device_hidden=desktop%2Bmobile%2Btablet%2Bconsole"
    "&multi-device=true"
    "&statType_hidden=social_media"
    "&region_hidden=ES"
    "&granularity=monthly"
    "&statType=Social%20Media"
    "&region=Spain"
    "&fromInt=201010"
    "&toInt=202510"
    "&fromMonthYear=2010-10"
    "&toMonthYear=2025-10"
    "&csv=1"
)

def build_url(geo_code: str, slug: str, name_in_url: str) -> str:
    """
    Replace country-specific parts of the template URL
    so that we can download monthly social-media share data
    for any European country.
    """
    url = template_url.replace("/spain/", f"/{slug}/")
    url = url.replace("region_hidden=ES", f"region_hidden={geo_code}")
    url = url.replace("region=Spain", f"region={name_in_url}")
    return url

## CELL 3: Download merged monthly social-media share panel

---------------------------------------------------------
This loops through all European countries, downloads their Statcounter
monthly CSV, parses it, adds country info, and concatenates everything
into a single panel.
The final file is saved under data/processed/

In [3]:

import io
import requests
import pandas as pd

all_frames = []

for geo, info in tries.items():
    slug = info["slug"]
    name_in_url = info["name_in_url"]

    # Build the country-specific Statcounter URL
    url = build_url(geo, slug, name_in_url)
    print(f"Downloading: {geo} → {slug}")

    # Download CSV content
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
    except Exception as e:
        print(f"❌ Connection error for {geo}: {e}")
        continue

    # Parse CSV
    try:
        df = pd.read_csv(io.StringIO(r.text))
        df["geo"] = geo
        df["country_name"] = name_in_url
        all_frames.append(df)
    except Exception as e:
        print(f"❌ CSV parsing failed for {geo}: {e}")

# Merge all countries into one dataframe
if all_frames:
    panel = pd.concat(all_frames, ignore_index=True)
    out_path = DATA_PROCESSED / "social_media_countries_monthly.csv"
    panel.to_csv(out_path, index=False)

    print("\n✅ Merged Statcounter file saved:", out_path)
    print("Total rows:", len(panel))
else:
    print("❌ No Statcounter data downloaded.")

Downloading: AL → albania
Downloading: AT → austria
Downloading: BA → bosnia-and-herzegovina
Downloading: BE → belgium
Downloading: BG → bulgaria
Downloading: CH → switzerland
Downloading: CY → cyprus
Downloading: CZ → czech-republic
Downloading: DE → germany
Downloading: DK → denmark
Downloading: EE → estonia
Downloading: EL → greece
Downloading: ES → spain
Downloading: FI → finland
Downloading: FR → france
Downloading: HR → croatia
Downloading: HU → hungary
Downloading: IE → ireland
Downloading: IS → iceland
Downloading: IT → italy
Downloading: LT → lithuania
Downloading: LU → luxembourg
Downloading: LV → latvia
Downloading: ME → montenegro
Downloading: MK → north-macedonia
Downloading: MT → malta
Downloading: NL → netherlands
Downloading: NO → norway
Downloading: PL → poland
Downloading: PT → portugal
Downloading: RO → romania
Downloading: RS → serbia
Downloading: SE → sweden
Downloading: SI → slovenia
Downloading: SK → slovakia
Downloading: TR → turkey
Downloading: UK → united-king

## CELL 4: Define RAW directory for Eurostat & OECD files

-------------------------------------------------------
All externally downloaded CSV files (Eurostat, OECD)
are expected to be stored under data/raw/.

In [22]:

import pandas as pd
import numpy as np
import os

DATA_RAW = DATA_RAW  # already defined in CELL 1
DATA_PROCESSED = DATA_PROCESSED  # already defined in CELL 1

print("RAW directory      :", DATA_RAW)
print("PROCESSED directory:", DATA_PROCESSED)

RAW directory      : /Users/ibrahimgozlukaya/Desktop/DSA210/data/raw
PROCESSED directory: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed


## CELL 5: Eurostat tin00127 — annual social media participation

-------------------------------------------------------------
Source: tin00127_linear_2_0.csv
We use:
- geo          : country code (ISO-2 style)
- TIME_PERIOD  : year (YYYY)
- OBS_VALUE    : share of individuals using social networks (% of individuals)

In [9]:

tin_path = DATA_RAW / "tin00127_linear_2_0.csv"
tin_raw = pd.read_csv(tin_path)

print("tin00127 raw shape:", tin_raw.shape)
# Clean and keep only the relevant columns
tin = tin_raw.rename(columns={
    "TIME_PERIOD": "year",
    "OBS_VALUE": "sm_participation",
})

# Ensure numeric year
tin["year"] = tin["year"].astype(int)

tin = tin[["geo", "year", "sm_participation"]].copy()

print("Clean tin shape:", tin.shape)

tin00127 raw shape: (424, 21)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,unit,Unit of measure,indic_is,Information society indicator,ind_type,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2018,NaN,48.33,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2019,NaN,52.13,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2020,NaN,54.80,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2021,NaN,60.77,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:TIN00127(1.0),Individuals using the internet for participati...,A,Annual,PC_IND,Percentage of individuals,I_IUSNET,Internet use: participating in social networks...,IND_TOTAL,...,AL,Albania,2022,NaN,64.07,NaN,NaN,NaN,NaN,NaN


Clean tin shape: (424, 3)


,geo,year,sm_participation
0,AL,2018,48.33
1,AL,2019,52.13
2,AL,2020,54.80
3,AL,2021,60.77
4,AL,2022,64.07


## CELL 6: Eurostat HICP — monthly index to inflation measures

-----------------------------------------------------------
Source: prc_hicp_midx__custom_19133241_linear.csv
Important columns:
- geo        : country name (e.g. "Austria", "Belgium", "Türkiye")
- TIME_PERIOD: monthly date string "YYYY-MM"
- OBS_VALUE  : HICP index (2015 = 100)

In [10]:

hicp_path = DATA_RAW / "prc_hicp_midx__custom_19134087_linear.csv"
hicp_raw = pd.read_csv(hicp_path)

print("HICP raw shape:", hicp_raw.shape)
# Rename to clearer names
hicp = hicp_raw.rename(columns={
    "geo": "geo_name",
    "TIME_PERIOD": "date",
    "OBS_VALUE": "hicp_index",
})

# Parse date and extract year/month
hicp["date"] = pd.to_datetime(hicp["date"])
hicp["year"] = hicp["date"].dt.year
hicp["month"] = hicp["date"].dt.month

print("Example country names in HICP:", sorted(hicp["geo_name"].unique())[:10])
# Map country names (in HICP) to Eurostat geo codes (used in tin00127 and unemployment)
name_to_geo = {
    "Albania": "AL",
    "Austria": "AT",
    "Belgium": "BE",
    "Bulgaria": "BG",
    "Croatia": "HR",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Denmark": "DK",
    "Estonia": "EE",
    "Finland": "FI",
    "France": "FR",
    "Germany": "DE",
    "Greece": "EL",
    "Hungary": "HU",
    "Iceland": "IS",
    "Ireland": "IE",
    "Italy": "IT",
    "Latvia": "LV",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Malta": "MT",
    "Montenegro": "ME",
    "Netherlands": "NL",
    "North Macedonia": "MK",
    "Norway": "NO",
    "Poland": "PL",
    "Portugal": "PT",
    "Romania": "RO",
    "Serbia": "RS",
    "Slovakia": "SK",
    "Slovenia": "SI",
    "Spain": "ES",
    "Sweden": "SE",
    "Switzerland": "CH",
    "Türkiye": "TR",
    "United Kingdom": "UK",
    # you can extend this dict if needed
}

hicp["geo"] = hicp["geo_name"].map(name_to_geo)

# Keep only rows where we have a valid geo code
hicp = hicp[~hicp["geo"].isna()].copy()

print("Clean HICP shape (with geo codes):", hicp.shape)
# (1) Monthly year-on-year inflation (for H3 monthly panel)
# --------------------------------------------------------
# Year-on-year inflation: percentage change compared to the same month one year earlier.

hicp = hicp.sort_values(["geo", "date"])
hicp["hicp_index_lag12"] = hicp.groupby("geo")["hicp_index"].shift(12)
hicp["inflation_yoy"] = (hicp["hicp_index"] / hicp["hicp_index_lag12"] - 1) * 100

hicp_monthly = hicp[["geo", "year", "month", "inflation_yoy"]].copy()

print("HICP monthly (yoy inflation) shape:", hicp_monthly.shape)
# (2) Annual inflation based on December-to-December changes (for H1/H2 annual panel)
# -----------------------------------------------------------------------------------
# Annual inflation = (December index this year / December index last year - 1) * 100

december = hicp[hicp["month"] == 12].copy()
december = december.sort_values(["geo", "year"])
december["inflation"] = december.groupby("geo")["hicp_index"].pct_change() * 100

inflation_annual = december[["geo", "year", "inflation"]].copy()

print("Annual inflation shape:", inflation_annual.shape)

HICP raw shape: (8636, 10)


,DATAFLOW,LAST UPDATE,freq,unit,coicop,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-01,105.58,NaN,NaN
1,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-02,105.90,NaN,NaN
2,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-03,106.86,NaN,NaN
3,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-04,107.15,NaN,NaN
4,ESTAT:PRC_HICP_MIDX(1.0),19/11/25 11:00:00,Monthly,"Index, 2005=100",All-items HICP,Austria,2008-05,107.72,NaN,NaN


Example country names in HICP: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Euro area (EA11-1999, EA12-2001, EA13-2007, EA15-2008, EA16-2009, EA17-2011, EA18-2014, EA19-2015, EA20-2023)', 'Euro area - 19 countries  (2015-2022)']
Clean HICP shape (with geo codes): (7003, 13)


,geo_name,geo,date,hicp_index
0,Austria,AT,2008-01-01,105.58
1,Austria,AT,2008-02-01,105.90
2,Austria,AT,2008-03-01,106.86
3,Austria,AT,2008-04-01,107.15
4,Austria,AT,2008-05-01,107.72


HICP monthly (yoy inflation) shape: (7003, 4)


,geo,year,month,inflation_yoy
0,AT,2008,1,NaN
1,AT,2008,2,NaN
2,AT,2008,3,NaN
3,AT,2008,4,NaN
4,AT,2008,5,NaN


Annual inflation shape: (556, 3)


,geo,year,inflation
11,AT,2008,NaN
23,AT,2009,1.051750
35,AT,2010,2.173713
47,AT,2011,3.398540
59,AT,2012,2.920663


## CELL 7: Eurostat unemployment — monthly rates

---------------------------------------------
Source: une_rt_m_linear_2_0.csv
We select:
- s_adj = "SA"       : seasonally adjusted
- age   = "TOTAL"    : total working-age population
- sex   = "T"        : both sexes
- unit  = "PC_ACT"   : percent of active population
Then we compute monthly series and annual averages.

In [11]:

unemp_path = DATA_RAW / "une_rt_m_linear_2_0.csv"
unemp_raw = pd.read_csv(unemp_path)

print("Unemployment raw shape:", unemp_raw.shape)

# 1) Filter to the standard unemployment rate definition
unemp_filtered = unemp_raw[
    (unemp_raw["s_adj"] == "SA") &        # seasonally adjusted
    (unemp_raw["age"]   == "TOTAL") &     # all ages
    (unemp_raw["sex"]   == "T") &         # both sexes
    (unemp_raw["unit"]  == "PC_ACT")      # % of active population
].copy()

print("Filtered unemployment shape:", unemp_filtered.shape)

# 2) Rename columns and parse dates
unemp_filtered = unemp_filtered.rename(columns={
    "TIME_PERIOD": "date",
    "OBS_VALUE": "unemployment",
})

unemp_filtered["date"] = pd.to_datetime(unemp_filtered["date"])
unemp_filtered["year"] = unemp_filtered["date"].dt.year
unemp_filtered["month"] = unemp_filtered["date"].dt.month

# 3) Monthly unemployment panel
unemp_monthly = (
    unemp_filtered[["geo", "year", "month", "unemployment"]]
    .sort_values(["geo", "year", "month"])
)

print("Unemployment monthly shape:", unemp_monthly.shape)

# 4) Annual average unemployment rate
unemp_annual = (
    unemp_monthly
    .groupby(["geo", "year"], as_index=False)["unemployment"]
    .mean()
)

print("Unemployment annual shape:", unemp_annual.shape)

Unemployment raw shape: (727293, 23)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,s_adj,Seasonal adjustment,age,Age class,unit,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
0,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-01,NaN,5.5,NaN,NaN,NaN,NaN,NaN
1,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-02,NaN,5.5,NaN,NaN,NaN,NaN,NaN
2,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-03,NaN,5.4,NaN,NaN,NaN,NaN,NaN
3,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-04,NaN,5.6,NaN,NaN,NaN,NaN,NaN
4,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,NSA,Unadjusted data (i.e. neither seasonally adjus...,TOTAL,Total,PC_ACT,...,AT,Austria,1994-05,NaN,5.6,NaN,NaN,NaN,NaN,NaN


Filtered unemployment shape: (14263, 23)


,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,freq,Time frequency,s_adj,Seasonal adjustment,age,Age class,unit,...,geo,Geopolitical entity (reporting),TIME_PERIOD,Time,OBS_VALUE,Observation value,OBS_FLAG,Observation status (Flag) V2 structure,CONF_STATUS,Confidentiality status (flag)
274526,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-01,NaN,4.0,NaN,NaN,NaN,NaN,NaN
274527,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-02,NaN,3.9,NaN,NaN,NaN,NaN,NaN
274528,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-03,NaN,3.9,NaN,NaN,NaN,NaN,NaN
274529,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-04,NaN,4.0,NaN,NaN,NaN,NaN,NaN
274530,dataflow,ESTAT:UNE_RT_M(1.0),Unemployment by sex and age - monthly data,M,Monthly,SA,"Seasonally adjusted data, not calendar adjuste...",TOTAL,Total,PC_ACT,...,AT,Austria,1995-05,NaN,4.1,NaN,NaN,NaN,NaN,NaN


Unemployment monthly shape: (14263, 4)


,geo,year,month,unemployment
274526,AT,1995,1,4.0
274527,AT,1995,2,3.9
274528,AT,1995,3,3.9
274529,AT,1995,4,4.0
274530,AT,1995,5,4.1


Unemployment annual shape: (1199, 3)


,geo,year,unemployment
0,AT,1995,4.241667
1,AT,1996,4.716667
2,AT,1997,4.758333
3,AT,1998,4.708333
4,AT,1999,4.141667


## CELL 8: OECD CCI export — wide to long, monthly & annual CCI

-------------------------------------------------------------
Source: export-2025-11-26T13_03_48.204Z.csv
This file is a wide table where:
- "Category" column contains dates ("2010-07-01 00:00:00")
- Each other column is a country or region
- Values use a comma as decimal separator and semicolon as delimiter

In [12]:

cci_path = DATA_RAW / "export-2025-11-26T13_03_48.204Z.csv"
cci_wide = pd.read_csv(
    cci_path,
    sep=";",        # semicolon-separated
    skiprows=2,     # skip the first two metadata rows
    decimal=",",    # use comma as decimal separator
)

print("CCI wide shape:", cci_wide.shape)
# Select only European country columns that we can map to Eurostat geo codes
cci_name_to_geo = {
    "Austria": "AT",
    "Belgium": "BE",
    "Czechia": "CZ",
    "Denmark": "DK",
    "Estonia": "EE",
    "Finland": "FI",
    "France": "FR",
    "Germany": "DE",
    "Greece": "EL",
    "Hungary": "HU",
    "Iceland": "IS",
    "Ireland": "IE",
    "Italy": "IT",
    "Latvia": "LV",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Netherlands": "NL",
    "Norway": "NO",
    "Poland": "PL",
    "Portugal": "PT",
    "Slovak Rep.": "SK",
    "Slovenia": "SI",
    "Spain": "ES",
    "Sweden": "SE",
    "Switzerland": "CH",
    "Türkiye": "TR",
    "United Kingdom": "UK",
}

europe_cols = ["Category"] + [name for name in cci_name_to_geo.keys() if name in cci_wide.columns]
cci_europe = cci_wide[europe_cols].copy()

print("CCI Europe subset shape:", cci_europe.shape)
cci_long = cci_europe.melt(
    id_vars="Category",
    var_name="country_name",
    value_name="cci",
)

# Map country names to Eurostat geo codes
cci_long["geo"] = cci_long["country_name"].map(cci_name_to_geo)
cci_long = cci_long[~cci_long["geo"].isna()].copy()

# Parse date and extract year/month
cci_long["date"] = pd.to_datetime(cci_long["Category"])
cci_long["year"] = cci_long["date"].dt.year
cci_long["month"] = cci_long["date"].dt.month

cci_monthly = cci_long[["geo", "year", "month", "cci"]].dropna()

print("CCI monthly shape:", cci_monthly.shape)
# Annual average CCI
cci_annual = (
    cci_monthly
    .groupby(["geo", "year"], as_index=False)["cci"]
    .mean()
)

print("CCI annual shape:", cci_annual.shape)

CCI wide shape: (184, 50)


,Category,Major five Asia economies,Australia,Austria,Belgium,Chile,Colombia,Costa Rica,Czechia,Denmark,...,European Union (since 2020),OECD,OECD excluding the euro area,OECD Europe,Brazil,China,India,Indonesia,Russia,South Africa
0,2010-07-01 00:00:00,100.16980,101.1949,100.2935,100.5633,101.5922,103.7792,102.9870,99.95223,101.1339,...,98.71791,99.25383,99.17513,99.47222,102.9224,99.73689,NaN,99.68826,101.2028,102.5821
1,2010-08-01 00:00:00,99.98219,101.5486,100.6558,100.8198,102.0630,104.2988,102.6625,99.65169,101.1359,...,99.14233,99.27330,99.09669,99.64468,103.0428,99.46487,NaN,99.80018,100.9577,102.6125
2,2010-09-01 00:00:00,99.70647,101.6113,100.8824,100.9553,102.6991,104.2489,102.3753,99.30165,101.1473,...,99.41579,99.29877,99.08424,99.73608,103.1625,99.05468,NaN,99.81923,100.8880,102.6349
3,2010-10-01 00:00:00,99.49446,101.5537,100.8755,101.0141,103.0448,103.7127,102.1615,98.98066,101.1613,...,99.52643,99.33994,99.15450,99.73548,103.2801,98.69066,NaN,99.87341,100.9000,102.6183
4,2010-11-01 00:00:00,99.32086,101.3013,100.6785,100.9837,102.9526,102.9605,102.0852,98.86246,101.1617,...,99.55367,99.41690,99.31138,99.70140,103.4245,98.33498,NaN,100.00740,100.8633,102.5609


CCI Europe subset shape: (184, 26)


,Category,Austria,Belgium,Czechia,Denmark,Estonia,Finland,France,Germany,Greece,...,Netherlands,Poland,Portugal,Slovak Rep.,Slovenia,Spain,Sweden,Switzerland,Türkiye,United Kingdom
0,2010-07-01 00:00:00,100.2935,100.5633,99.95223,101.1339,100.1657,101.9947,98.55846,100.7156,96.30705,...,100.0866,100.13370,97.70040,99.98700,98.21307,95.97453,102.1055,102.0776,99.98065,99.27699
1,2010-08-01 00:00:00,100.6558,100.8198,99.65169,101.1359,100.3743,102.3227,98.78314,101.2009,96.39933,...,100.1937,99.90450,97.67054,99.72249,98.07329,96.56970,102.1560,102.0857,100.15990,99.08302
2,2010-09-01 00:00:00,100.8824,100.9553,99.30165,101.1473,100.4359,102.4197,98.98823,101.5459,96.24638,...,100.2077,99.64635,97.25994,99.48306,98.13566,97.02423,102.1740,101.9900,100.49300,98.73116
3,2010-10-01 00:00:00,100.8755,101.0141,98.98066,101.1613,100.3462,102.1022,99.23347,101.7402,96.01804,...,100.2202,99.46078,96.24286,99.31108,98.30521,97.09800,102.0650,101.8351,100.71870,98.41450
4,2010-11-01 00:00:00,100.6785,100.9837,98.86246,101.1617,100.4737,101.5863,99.38045,101.8713,95.75428,...,100.2195,99.35139,95.59222,99.37595,98.26225,96.90553,101.9832,101.6675,101.00500,98.11409


CCI monthly shape: (4598, 4)


,geo,year,month,cci
0,AT,2010,7,100.2935
1,AT,2010,8,100.6558
2,AT,2010,9,100.8824
3,AT,2010,10,100.8755
4,AT,2010,11,100.6785


CCI annual shape: (400, 3)


,geo,year,cci
0,AT,2010,100.650433
1,AT,2011,99.653852
2,AT,2012,99.095582
3,AT,2013,99.597598
4,AT,2014,99.524171


## CELL 9: Statcounter merged monthly file — platform shares + ent/prof

--------------------------------------------------------------------
Source: data/processed/social_media_countries_monthly.csv
This file was created in earlier cells by downloading
monthly social-media market shares for each country.

In [13]:

sc_path = DATA_PROCESSED / "social_media_countries_monthly.csv"
sc_raw = pd.read_csv(sc_path)

print("Statcounter raw shape:", sc_raw.shape)
# Identify the date column (usually "Month" or "Date")
date_col_candidates = [
    c for c in sc_raw.columns
    if c.lower().startswith("date") or c.lower().startswith("month")
]
print("Date column candidates:", date_col_candidates)

date_col = date_col_candidates[0]
print("Using date column:", date_col)

# Parse date and extract year/month
sc_raw["date"] = pd.to_datetime(sc_raw[date_col])
sc_raw["year"] = sc_raw["date"].dt.year
sc_raw["month"] = sc_raw["date"].dt.month
# Identify platform columns
candidate_platforms = ["Facebook", "Instagram", "YouTube", "TikTok", "LinkedIn"]
platform_cols = [c for c in candidate_platforms if c in sc_raw.columns]

print("Platform columns found:", platform_cols)

# Keep only geo, year, month, and platform shares
sc_monthly = sc_raw[["geo", "year", "month"] + platform_cols].copy()

# Compute entertainment vs professional shares
ent_cols = [c for c in ["Facebook", "Instagram", "YouTube", "TikTok"] if c in sc_monthly.columns]
prof_cols = [c for c in ["LinkedIn"] if c in sc_monthly.columns]

print("Entertainment platforms:", ent_cols)
print("Professional platforms:", prof_cols)

if ent_cols:
    sc_monthly["ent_share"] = sc_monthly[ent_cols].sum(axis=1)
else:
    sc_monthly["ent_share"] = np.nan

if prof_cols:
    sc_monthly["prof_share"] = sc_monthly[prof_cols].sum(axis=1)
else:
    sc_monthly["prof_share"] = np.nan

sc_monthly["ent_minus_prof"] = sc_monthly["ent_share"] - sc_monthly["prof_share"]

print("Clean Statcounter monthly shape:", sc_monthly.shape)

Statcounter raw shape: (6878, 31)


,Date,Facebook,Twitter,YouTube,Pinterest,Instagram,StumbleUpon,Tumblr,reddit,LinkedIn,...,Odnoklassniki,orkut,Fark,MySpace,Unnamed: 1,Tuenti,Netlog,FriendFeed,Hi5,Hyves
0,2010-10,86.40,0.50,11.36,0.0,0.0,1.15,0.0,0.08,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-11,87.67,0.43,10.63,0.0,0.0,0.83,0.0,0.07,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-12,87.53,0.37,10.80,0.0,0.0,0.89,0.0,0.06,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01,90.20,0.26,8.28,0.0,0.0,0.94,0.0,0.06,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-02,88.35,0.37,9.84,0.0,0.0,1.14,0.0,0.04,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Date column candidates: ['Date']
Using date column: Date
Platform columns found: ['Facebook', 'Instagram', 'YouTube', 'LinkedIn']
Entertainment platforms: ['Facebook', 'Instagram', 'YouTube']
Professional platforms: ['LinkedIn']
Clean Statcounter monthly shape: (6878, 10)


,geo,year,month,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof
0,AL,2010,10,86.40,0.0,11.36,0.03,97.76,0.03,97.73
1,AL,2010,11,87.67,0.0,10.63,0.03,98.30,0.03,98.27
2,AL,2010,12,87.53,0.0,10.80,0.04,98.33,0.04,98.29
3,AL,2011,1,90.20,0.0,8.28,0.05,98.48,0.05,98.43
4,AL,2011,2,88.35,0.0,9.84,0.05,98.19,0.05,98.14


## CELL 10: Build monthly H3 panel

-------------------------------
This panel combines:
- Statcounter monthly platform shares (ent/prof)
- Monthly inflation (HICP, yoy)
- Monthly unemployment
- Monthly CCI
It is used to test H3 at the monthly level.

In [14]:

panel_monthly_h3 = sc_monthly.copy()

panel_monthly_h3 = (
    panel_monthly_h3
    .merge(hicp_monthly, on=["geo", "year", "month"], how="left")
    .merge(unemp_monthly, on=["geo", "year", "month"], how="left")
    .merge(cci_monthly, on=["geo", "year", "month"], how="left")
)

print("Monthly H3 panel shape:", panel_monthly_h3.shape)
# Save monthly H3 panel
out_monthly_h3 = DATA_PROCESSED / "panel_monthly_h3.csv"
panel_monthly_h3.to_csv(out_monthly_h3, index=False)

print("✅ Monthly H3 panel saved to:", out_monthly_h3)

Monthly H3 panel shape: (6878, 13)


,geo,year,month,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof,inflation_yoy,unemployment,cci
0,AL,2010,10,86.40,0.0,11.36,0.03,97.76,0.03,97.73,NaN,NaN,NaN
1,AL,2010,11,87.67,0.0,10.63,0.03,98.30,0.03,98.27,NaN,NaN,NaN
2,AL,2010,12,87.53,0.0,10.80,0.04,98.33,0.04,98.29,NaN,NaN,NaN
3,AL,2011,1,90.20,0.0,8.28,0.05,98.48,0.05,98.43,NaN,NaN,NaN
4,AL,2011,2,88.35,0.0,9.84,0.05,98.19,0.05,98.14,NaN,NaN,NaN


✅ Monthly H3 panel saved to: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed/panel_monthly_h3.csv


## CELL 11: Statcounter annual summary

-----------------------------------
We aggregate Statcounter platform shares to the annual level
for use in the annual panel (H1/H2). We keep:
- average platform shares per year
- average ent_share, prof_share, ent_minus_prof per year

In [15]:

sc_annual = (
    sc_monthly
    .groupby(["geo", "year"], as_index=False)[platform_cols + ["ent_share", "prof_share", "ent_minus_prof"]]
    .mean()
)

print("Statcounter annual shape:", sc_annual.shape)

Statcounter annual shape: (608, 9)


,geo,year,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof
0,AL,2010,87.200000,0.0,10.930000,0.033333,98.130000,0.033333,98.096667
1,AL,2011,89.330833,0.0,7.629167,0.073333,96.960000,0.073333,96.886667
2,AL,2012,94.013333,0.0,3.511667,0.069167,97.525000,0.069167,97.455833
3,AL,2013,93.751667,0.0,1.822500,0.046667,95.574167,0.046667,95.527500
4,AL,2014,94.268333,0.0,0.569167,0.035000,94.837500,0.035000,94.802500


## CELL 12: Build final annual panel

---------------------------------
This panel combines:
- Annual social-media participation (tin00127)
- Annual inflation (HICP December-to-December)
- Annual unemployment (average)
- Annual CCI (average)
- Annual Statcounter summary (platform + ent/prof shares)

In [16]:

panel_annual = tin.copy()

panel_annual = (
    panel_annual
    .merge(inflation_annual, on=["geo", "year"], how="left")
    .merge(unemp_annual, on=["geo", "year"], how="left")
    .merge(cci_annual, on=["geo", "year"], how="left")
    .merge(sc_annual, on=["geo", "year"], how="left")
)

print("Final annual panel shape:", panel_annual.shape)
# Save final annual panel
out_annual = DATA_PROCESSED / "panel_annual.csv"
panel_annual.to_csv(out_annual, index=False)

print("✅ Annual panel saved to:", out_annual)

Final annual panel shape: (424, 13)


,geo,year,sm_participation,inflation,unemployment,cci,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof
0,AL,2018,48.33,NaN,NaN,NaN,79.240833,3.094167,7.065000,0.298333,89.400000,0.298333,89.101667
1,AL,2019,52.13,NaN,NaN,NaN,69.003333,5.556667,4.623333,0.275833,79.183333,0.275833,78.907500
2,AL,2020,54.80,NaN,NaN,NaN,75.620000,6.143333,3.115000,0.157500,84.878333,0.157500,84.720833
3,AL,2021,60.77,NaN,NaN,NaN,91.660833,1.360000,1.035833,0.115833,94.056667,0.115833,93.940833
4,AL,2022,64.07,NaN,NaN,NaN,93.218333,1.830000,1.087500,0.140000,96.135833,0.140000,95.995833


✅ Annual panel saved to: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed/panel_annual.csv


In [42]:
# =====================================================================
# FINAL MONTHLY PANEL PIPELINE (WITH GEO MAPPING + DATE FIX)
# Produces: data/processed/panel_monthly_h3.csv
# =====================================================================

import pandas as pd
import numpy as np
from pathlib import Path

# -------------------------------
# Paths
# -------------------------------
PROJECT_ROOT = Path.home() / "Desktop" / "DSA210"
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print("RAW:", DATA_RAW)
print("PROCESSED:", DATA_PROCESSED)


# ==========================================================
# 1) MONTHLY UNEMPLOYMENT (Eurostat)
# ==========================================================
unemp = pd.read_csv(DATA_RAW / "une_rt_m_linear_2_0.csv")

unemp = unemp.rename(columns={
    "TIME_PERIOD": "date",
    "OBS_VALUE": "unemployment",
})

# Fix date → month-level timestamp (1st day of month)
unemp["date"] = pd.to_datetime(unemp["date"]).dt.to_period("M").dt.to_timestamp()

unemp["geo"] = unemp["geo"].str.upper()
unemp["year"] = unemp["date"].dt.year
unemp["month"] = unemp["date"].dt.month

unemp_m = unemp[["geo", "year", "month", "unemployment"]].copy()
print("Unemployment monthly rows:", len(unemp_m))


# ==========================================================
# 2) MONTHLY INFLATION (HICP FROM EUROSTAT) — GEO FIX
# ==========================================================
hicp = pd.read_csv(DATA_RAW / "prc_hicp_midx__custom_19134087_linear.csv")

# Normalize column names
hicp.columns = [c.lower().strip() for c in hicp.columns]

# Auto-detect main columns
date_col  = next(c for c in hicp.columns if "time" in c)
value_col = next(c for c in hicp.columns if "value" in c)
geo_col   = next(c for c in hicp.columns if "geo" in c)

hicp = hicp.rename(columns={
    date_col: "date",
    value_col: "hicp_index",
    geo_col: "geo",
})

# Convert geo to UPPERCASE to match mapping keys
hicp["geo"] = hicp["geo"].str.upper()

# Map country names → ISO-2 codes to match Statcounter
geo_map = {
    "ALBANIA": "AL",
    "AUSTRIA": "AT",
    "BELGIUM": "BE",
    "BOSNIA AND HERZEGOVINA": "BA",
    "BULGARIA": "BG",
    "CROATIA": "HR",
    "CYPRUS": "CY",
    "CZECHIA": "CZ",
    "CZECH REPUBLIC": "CZ",
    "DENMARK": "DK",
    "ESTONIA": "EE",
    "FINLAND": "FI",
    "FRANCE": "FR",
    "GERMANY": "DE",
    "GREECE": "EL",
    "HUNGARY": "HU",
    "ICELAND": "IS",
    "IRELAND": "IE",
    "ITALY": "IT",
    "LATVIA": "LV",
    "LITHUANIA": "LT",
    "LUXEMBOURG": "LU",
    "MALTA": "MT",
    "MONTENEGRO": "ME",
    "NETHERLANDS": "NL",
    "NORTH MACEDONIA": "MK",
    "NORWAY": "NO",
    "POLAND": "PL",
    "PORTUGAL": "PT",
    "ROMANIA": "RO",
    "SERBIA": "RS",
    "SLOVAKIA": "SK",
    "SLOVENIA": "SI",
    "SPAIN": "ES",
    "SWEDEN": "SE",
    "SWITZERLAND": "CH",
    "TURKEY": "TR",
    "UNITED KINGDOM": "UK",
    "KOSOVO*": "XK",
}

hicp["geo"] = hicp["geo"].map(geo_map)

# Drop EU aggregates (they map to NaN) and keep only real countries
hicp = hicp[hicp["geo"].notna()]

# Fix date to month-level timestamp
hicp["date"] = pd.to_datetime(hicp["date"]).dt.to_period("M").dt.to_timestamp()
hicp["year"] = hicp["date"].dt.year
hicp["month"] = hicp["date"].dt.month

hicp_m = hicp[["geo", "year", "month", "hicp_index"]].copy()
print("HICP monthly rows (after GEO fix):", len(hicp_m))


# ==========================================================
# 3) STATCOUNTER MONTHLY DATA (platform shares)
# ==========================================================
sc = pd.read_csv(DATA_RAW / "social_media_countries_monthly.csv")

# Auto-detect date + geo columns
date_col = next(c for c in sc.columns if "date" in c.lower() or "month" in c.lower())
geo_col  = next(c for c in sc.columns if c.lower() in ["geo", "country", "region"])

# Detect platform columns
platform_map = {}
for c in sc.columns:
    lc = c.lower()
    if "facebook" in lc:
        platform_map["facebook"] = c
    if "instagram" in lc:
        platform_map["instagram"] = c
    if "youtube" in lc:
        platform_map["youtube"] = c
    if "linkedin" in lc:
        platform_map["linkedin"] = c

# Standardize column names
sc = sc.rename(columns={
    date_col: "date",
    geo_col: "geo",
    platform_map["facebook"]: "facebook",
    platform_map["instagram"]: "instagram",
    platform_map["youtube"]: "youtube",
    platform_map["linkedin"]: "linkedin",
})

# Fix date
sc["date"] = pd.to_datetime(sc["date"]).dt.to_period("M").dt.to_timestamp()
sc["geo"] = sc["geo"].str.upper()
sc["year"] = sc["date"].dt.year
sc["month"] = sc["date"].dt.month

sc_m = sc[["geo", "year", "month", "facebook", "instagram", "youtube", "linkedin"]].copy()
print("Statcounter monthly rows:", len(sc_m))


# ==========================================================
# 4) MERGE ALL MONTHLY DATA
# ==========================================================
panel_m = (
    sc_m
    .merge(unemp_m, on=["geo", "year", "month"], how="left")
    .merge(hicp_m, on=["geo", "year", "month"], how="left")
    .sort_values(["geo", "year", "month"])
)

print("\nMerged monthly panel shape:", panel_m.shape)


# ==========================================================
# 5) PLATFORM CATEGORIES (entertainment vs professional)
# ==========================================================
panel_m["ent_share"]  = panel_m[["instagram", "facebook", "youtube"]].mean(axis=1)
panel_m["prof_share"] = panel_m["linkedin"]


# ==========================================================
# 6) MONTHLY CHANGE VARIABLES (Δ) — H3
# ==========================================================
panel_m["d_unemp"] = panel_m.groupby("geo")["unemployment"].diff()
panel_m["d_infl"]  = panel_m.groupby("geo")["hicp_index"].diff()
panel_m["d_ent"]   = panel_m.groupby("geo")["ent_share"].diff()
panel_m["d_prof"]  = panel_m.groupby("geo")["prof_share"].diff()

print("\nNon-missing d_infl:", panel_m["d_infl"].notna().sum())
print("Non-missing d_ent :", panel_m["d_ent"].notna().sum())


# ==========================================================
# 7) SAVE FINAL MONTHLY PANEL
# ==========================================================
output_path = DATA_PROCESSED / "panel_monthly_h3.csv"
panel_m.to_csv(output_path, index=False)

print("\n✓ Saved final monthly panel to:", output_path)

RAW: /Users/ibrahimgozlukaya/Desktop/DSA210/data/raw
PROCESSED: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed
Unemployment monthly rows: 727293
HICP monthly rows (after GEO fix): 6789
Statcounter monthly rows: 6878

Merged monthly panel shape: (306593, 9)

Non-missing d_infl: 294348
Non-missing d_ent : 290295

✓ Saved final monthly panel to: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed/panel_monthly_h3.csv


,geo,year,month,facebook,instagram,youtube,linkedin,unemployment,hicp_index,ent_share,prof_share,d_unemp,d_infl,d_ent,d_prof
0,AL,2010,10,86.40,0.0,11.36,0.03,NaN,NaN,32.586667,0.03,NaN,NaN,NaN,NaN
1,AL,2010,11,87.67,0.0,10.63,0.03,NaN,NaN,32.766667,0.03,NaN,NaN,0.180000,0.00
2,AL,2010,12,87.53,0.0,10.80,0.04,NaN,NaN,32.776667,0.04,NaN,NaN,0.010000,0.01
3,AL,2011,1,90.20,0.0,8.28,0.05,NaN,NaN,32.826667,0.05,NaN,NaN,0.050000,0.01
4,AL,2011,2,88.35,0.0,9.84,0.05,NaN,NaN,32.730000,0.05,NaN,NaN,-0.096667,0.00
5,AL,2011,3,91.27,0.0,7.27,0.05,NaN,NaN,32.846667,0.05,NaN,NaN,0.116667,0.00
6,AL,2011,4,92.66,0.0,6.11,0.04,NaN,NaN,32.923333,0.04,NaN,NaN,0.076667,-0.01
7,AL,2011,5,93.64,0.0,4.91,0.10,NaN,NaN,32.850000,0.10,NaN,NaN,-0.073333,0.06
8,AL,2011,6,88.08,0.0,9.10,0.08,NaN,NaN,32.393333,0.08,NaN,NaN,-0.456667,-0.02
9,AL,2011,7,84.16,0.0,11.46,0.10,NaN,NaN,31.873333,0.10,NaN,NaN,-0.520000,0.02


In [ ]:
# =====================================================================
# FINAL ANNUAL PANEL (panel_a FINAL) — stress + ΔSM
# Saved directly to data/processed for EDA use.
# =====================================================================

import pandas as pd
import numpy as np
from pathlib import Path

# -------------------------------
# 0) Paths
# -------------------------------
PROJECT_ROOT = Path.home() / "Desktop" / "DSA210"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

# -------------------------------
# 1) Load base annual panel
# -------------------------------
panel_a = pd.read_csv(DATA_PROCESSED / "panel_annual.csv")

print("Loaded base annual panel.")
print("Shape:", panel_a.shape)
print("Columns:", panel_a.columns.tolist())

# -------------------------------
# 2) Sort for ΔSM
# -------------------------------
panel_a = panel_a.sort_values(["geo", "year"]).copy()

# -------------------------------
# 3) ΔSM (yearly change in participation)
# -------------------------------
panel_a["d_sm"] = panel_a.groupby("geo")["sm_participation"].diff()

# -------------------------------
# 4) Remove rows where stress inputs missing
# -------------------------------
df = panel_a.dropna(subset=["inflation", "unemployment", "cci", "sm_participation"]).copy()

# -------------------------------
# 5) Compute Z-scores
# -------------------------------
df["infl_z"]  = (df["inflation"]    - df["inflation"].mean())    / df["inflation"].std()
df["unemp_z"] = (df["unemployment"] - df["unemployment"].mean()) / df["unemployment"].std()
df["cci_z"]   = (df["cci"]          - df["cci"].mean())          / df["cci"].std()

# -------------------------------
# 6) Stress Index
# Formula: stress = z(inflation) + z(unemployment) − z(CCI)
# -------------------------------
df["stress"] = df["infl_z"] + df["unemp_z"] - df["cci_z"]

print("\nStress index created.")
print("Unique stress values:", df["stress"].nunique())

# -------------------------------
# 7) Merge stress back into main annual panel
# -------------------------------
stress_cols = df[["geo", "year", "stress"]].copy()
panel_a = panel_a.merge(stress_cols, on=["geo", "year"], how="left")

# -------------------------------
# 8) Save final annual panel
# -------------------------------
out_path = DATA_PROCESSED / "panel_annual_final.csv"
panel_a.to_csv(out_path, index=False)

print("\n✓ FINAL annual panel saved!")
print("File:", out_path)
print("Final shape:", panel_a.shape)

Loaded base annual panel.
Shape: (424, 13)
Columns: ['geo', 'year', 'sm_participation', 'inflation', 'unemployment', 'cci', 'Facebook', 'Instagram', 'YouTube', 'LinkedIn', 'ent_share', 'prof_share', 'ent_minus_prof']

Stress index created.
Unique stress values: 286

✓ FINAL annual panel saved!
File: /Users/ibrahimgozlukaya/Desktop/DSA210/data/processed/panel_annual_final.csv
Final shape: (424, 15)


,geo,year,sm_participation,inflation,unemployment,cci,Facebook,Instagram,YouTube,LinkedIn,ent_share,prof_share,ent_minus_prof,d_sm,stress
0,AL,2018,48.33,NaN,NaN,NaN,79.240833,3.094167,7.065000,0.298333,89.400000,0.298333,89.101667,NaN,NaN
1,AL,2019,52.13,NaN,NaN,NaN,69.003333,5.556667,4.623333,0.275833,79.183333,0.275833,78.907500,3.80,NaN
2,AL,2020,54.80,NaN,NaN,NaN,75.620000,6.143333,3.115000,0.157500,84.878333,0.157500,84.720833,2.67,NaN
3,AL,2021,60.77,NaN,NaN,NaN,91.660833,1.360000,1.035833,0.115833,94.056667,0.115833,93.940833,5.97,NaN
4,AL,2022,64.07,NaN,NaN,NaN,93.218333,1.830000,1.087500,0.140000,96.135833,0.140000,95.995833,3.30,NaN
5,AL,2023,66.27,NaN,NaN,NaN,92.633333,4.280000,0.498333,0.065833,97.411667,0.065833,97.345833,2.20,NaN
6,AL,2024,68.95,NaN,NaN,NaN,92.475833,4.272500,0.352500,0.115000,97.100833,0.115000,96.985833,2.68,NaN
7,AT,2013,37.45,1.982211,5.708333,99.597598,78.099167,0.000000,4.810833,0.131667,82.910000,0.131667,82.778333,NaN,-0.457132
8,AT,2014,44.24,0.805715,6.050000,99.524171,83.270000,0.000000,0.965000,0.038333,84.235000,0.038333,84.196667,6.79,-0.503133
9,AT,2015,45.33,1.071193,6.158333,99.705362,83.950833,0.000000,0.149167,0.041667,84.100000,0.041667,84.058333,1.09,-0.534838


,geo,year,sm_participation,d_sm,stress
0,AL,2018,48.33,NaN,NaN
1,AL,2019,52.13,3.80,NaN
2,AL,2020,54.80,2.67,NaN
3,AL,2021,60.77,5.97,NaN
4,AL,2022,64.07,3.30,NaN
5,AL,2023,66.27,2.20,NaN
6,AL,2024,68.95,2.68,NaN
7,AT,2013,37.45,NaN,-0.457132
8,AT,2014,44.24,6.79,-0.503133
9,AT,2015,45.33,1.09,-0.534838
